In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

from feature_extractor import FeatureExtractor

In [2]:
# load old feature_extraction file:
anno_features = pickle.load(open('../../pvc-meteor/features/extracted_features_06-01-2023-18-39.pkl', 'rb'))

In [ ]:
extractor = FeatureExtractor(15)
out_dict = dict()

for name in tqdm(anno_features['annotations'].keys(), smoothing=0):
    anno_dict = extractor.annotation_processor(name)
    out_dict.update({name: anno_dict})

with open('../../pvc-meteor/features/annotations.pickle', 'wb') as file:
    pickle.dump(out_dict, file)

  1%|          | 15/1213 [00:24<32:40,  1.64s/it]

## get label distribution

In [ ]:
out_dict['REC_2020_10_12_03_08_38_F.MP4']['feature_length']

In [ ]:
total_frames = 0
cat_array = np.zeros((7,))
unannoted_frames = 0

for name in tqdm(out_dict.keys()):
    total_frames += out_dict[name]['feature_length']
    annotation = out_dict[name]['anno']
    new_cat = np.sum(annotation, axis=0)
    unannoted_frames += np.any(annotation, axis=1).sum()

    cat_array = cat_array + new_cat


In [ ]:
cat_list = cat_array.tolist()
cat_list[2] - total_frames

In [ ]:
cat_list, total_frames, unannoted_frames